In [ ]:
# Section 007 Group 02 Basketball Player Position Classification
# This project will include KNN classification for different basketball player positions based on player's physical and game statstics
# That being said, the algorithm will be able to predict a basketball player just entering any parameters

In [2]:
### The necessary Libraries required
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [5]:
player_data <- read_csv("data/player_data.csv")
season_data <- read_csv("data/Seasons_Stats.csv")

player_data
season_stats

Parsed with column specification:
cols(
  name = col_character(),
  year_start = col_double(),
  year_end = col_double(),
  position = col_character(),
  height = col_character(),
  weight = col_double(),
  birth_date = col_character(),
  college = col_character()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_double(),
  Player = col_character(),
  Pos = col_character(),
  Tm = col_character(),
  GS = col_logical(),
  `3PAr` = col_logical(),
  `ORB%` = col_logical(),
  `DRB%` = col_logical(),
  `TRB%` = col_logical(),
  `AST%` = col_logical(),
  `STL%` = col_logical(),
  `BLK%` = col_logical(),
  `TOV%` = col_logical(),
  `USG%` = col_logical(),
  blanl = col_logical(),
  blank2 = col_logical(),
  OBPM = col_logical(),
  DBPM = col_logical(),
  BPM = col_logical(),
  VORP = col_logical(),
  `3P` = col_logical()
  # ... with 7 more columns
)

See spec(...) for full column specifications.

Warning message:
“394102 

name,year_start,year_end,position,height,weight,birth_date,college
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
Alaa Abdelnaby,1991,1995,F-C,6-10,240,"June 24, 1968",Duke University
Zaid Abdul-Aziz,1969,1978,C-F,6-9,235,"April 7, 1946",Iowa State University
Kareem Abdul-Jabbar,1970,1989,C,7-2,225,"April 16, 1947","University of California, Los Angeles"
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Bill Zopf,1971,1971,G,6-1,170,"June 7, 1948",Duquesne University
Ivica Zubac,2017,2018,C,7-1,265,"March 18, 1997",NA
Matt Zunic,1949,1949,G-F,6-3,195,"December 19, 1919",George Washington University


ERROR: Error in eval(expr, envir, enclos): object 'season_stats' not found


In [6]:
## Now that we loaded our stats the next step is to wrangle. There is a bunch of information that is not required in the knn analysis

In [31]:
#Wrangling
#We only want players in 2017, this is the most current reported data

# player_data_2017 <- filter(player_data, Year == "2017")
# player_data_2017

season_data_2017 <- filter(season_data, Year == "2017") %>%
    group_by(Player)
season_data_2017


#We want the predicted class: Pos 
#We want 


# combined_data_set <- select(season_stats, ) %>%
#     select()


X1,Year,Player,Pos,Age,Tm,G,GS,MP,PER,⋯,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<lgl>,<dbl>,<dbl>,⋯,<dbl>,<lgl>,<lgl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
24096,2017,Alex Abrines,SG,23,OKC,68,NA,1055,10.1,⋯,0.898,NA,NA,86,40,NA,NA,NA,114,406
24097,2017,Quincy Acy,PF,26,TOT,38,TRUE,558,11.8,⋯,0.750,NA,NA,115,18,NA,NA,NA,67,222
24098,2017,Quincy Acy,PF,26,DAL,6,FALSE,48,-1.4,⋯,0.667,NA,NA,8,0,FALSE,FALSE,NA,9,13
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
24688,2017,Stephen Zimmerman,C,20,ORL,19,FALSE,108,7.3,⋯,0.600,NA,NA,35,4,NA,NA,NA,17,23
24689,2017,Paul Zipser,SF,22,CHI,44,NA,843,6.9,⋯,0.775,NA,NA,125,36,NA,NA,NA,78,240
24690,2017,Ivica Zubac,C,19,LAL,38,NA,609,17.0,⋯,0.653,NA,NA,159,30,NA,NA,NA,66,284


In [32]:
#convert wanted parameters to class
season_data_2017 <- season_data_2017 %>%
    mutate(Player = as_factor(Player)) %>%
    mutate(PPG = PTS/G) %>%
    mutate(APG = AST/G) %>%
    mutate(RPG = TRB/G) %>%
    mutate(SPG = STL/G) %>% #just for experimentation
    mutate(BPG = BLK/G) %>%

    #select(Player, PPG, APG, RPG, SPG, BPG, "3P%","2P%", "FT%", "FG%" ) %>%
    summarize(across(na.rm = TRUE))
season_data_2017

`summarise()` regrouping output by 'Player' (override with `.groups` argument)



Player,X1,Year,Pos,Age,Tm,G,GS,MP,PER,⋯,STL,BLK,TOV,PF,PTS,PPG,APG,RPG,SPG,BPG
<fct>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<lgl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A.J. Hammons,24304,2017,C,24,DAL,22,FALSE,163,8.4,⋯,TRUE,NA,NA,21,48,2.181818,0.1818182,1.636364,0.04545455,NA
Aaron Brooks,24165,2017,PG,32,IND,65,FALSE,894,9.5,⋯,NA,NA,NA,93,322,4.953846,1.9230769,1.061538,NA,NA
Aaron Gordon,24289,2017,SF,21,ORL,80,NA,2298,14.4,⋯,NA,NA,NA,172,1019,12.737500,1.8750000,5.062500,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Zach LaVine,24408,2017,SG,21,MIN,47,NA,1749,14.6,⋯,NA,NA,NA,104,889,18.914894,2.957447,3.382979,NA,NA
Zach Randolph,24550,2017,PF,35,MEM,73,NA,1786,18.5,⋯,NA,NA,NA,136,1028,14.082192,1.671233,8.191781,NA,NA
Zaza Pachulia,24515,2017,C,32,GSW,70,NA,1268,16.1,⋯,NA,NA,NA,166,426,6.085714,1.885714,5.857143,NA,NA


In [3]:
# sample graph
ggplot()
    